# Optimization of the Cross Section of a Beam

In [1]:
L = 100.  # length of the console
w, h = 10., 10.  # width and height of the rectangular cross section
F = -1000.  # value of the vertical load at the free end
E = 210000.0  # Young's modulus
nu = 0.3  # Poisson's ratio


In [2]:
from sigmaepsilon.solid import Structure, LineMesh, PointData
from neumann.linalg import linspace, Vector
from polymesh.space import StandardFrame, PointCloud, frames_of_lines
from sigmaepsilon.solid.fem.cells import B2 as Beam

import numpy as np
from numpy import pi as PI

# space
GlobalFrame = StandardFrame(dim=3)

# mesh
nElem = 5  # number of finite elements to use
p0 = np.array([0., 0., 0.])
p1 = np.array([L, 0., 0.])
coords = linspace(p0, p1, nElem+1)
coords = PointCloud(coords, frame=GlobalFrame).show()
topo = np.zeros((nElem, 2), dtype=int)
topo[:, 0] = np.arange(nElem)
topo[:, 1] = np.arange(nElem) + 1

# support at the leftmost, load at the rightmost node
loads = np.zeros((coords.shape[0], 6))
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
global_load_vector = Vector([0., 0, F], frame=GlobalFrame).show()
loads[-1, :3] = global_load_vector
fixity[0, :] = True

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, frames=frames)

# set up mesh and structure
def build(model:np.ndarray):
    mesh = LineMesh(pd, cd, model=model, frame=GlobalFrame)
    structure = Structure(mesh=mesh)
    return structure

def linsolve(structure:Structure):
    structure.linsolve()
    return structure

def postproc(structure:Structure):    
    dofsol = structure.nodal_dof_solution()[:, :3]
    return dofsol[-1, :3]

In [3]:
# cross section
A = w * h  # area
Iy = w * h**3 / 12  # second moment of inertia around the y axis
Iz = w * h**3 / 12  # second moment of inertia around the z axis
Ix = Iy + Iz  # torsional inertia

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

d = postproc(linsolve(build(Hooke)))

In [4]:
from neumann.optimize import BinaryGeneticAlgorithm

Vmax = w * h * L
Wmax = np.abs(global_load_vector @ d)

# Bernoulli solution
def objective_analytic(x):
    w, h = x
    A = w * h  # area
    Iy = w * h**3 / 12  # second moment of inertia around the y axis
    EI = E * Iy
    sol_exact = F * L**3 / (3 * EI)
    res = np.abs(sol_exact * F)
    
    # check volume constraint
    V = A * L
    if V > Vmax:
        res *= 1e10
        
    return res
    
    
def objective_fem(x):
    w, h = x
    A = w * h  # area
    Iy = w * h**3 / 12  # second moment of inertia around the y axis
    Iz = w * h**3 / 12  # second moment of inertia around the z axis
    Ix = Iy + Iz  # torsional inertia

    # model stiffness matrix
    G = E / (2 * (1 + nu))
    Hooke = np.array([
        [E*A, 0, 0, 0],
        [0, G*Ix, 0, 0],
        [0, 0, E*Iy, 0],
        [0, 0, 0, E*Iz]
    ])

    d = postproc(linsolve(build(Hooke)))
    res = np.abs(global_load_vector @ d)
    
    # check volume constraint
    V = A * L
    if V > Vmax:
        res *= 1e10
    
    return res

In [5]:
ranges = [[5., 15.], [5., 15.]]
BGA = BinaryGeneticAlgorithm(objective_analytic, ranges, length=12, nPop=100, maxiter=100)
x = BGA.solve()
x = BGA.best_phenotype()
fx_analytic = objective_analytic(x)
x, fx_analytic

(array([ 6.65811966, 14.995116  ]), 848.4760966743787)

In [6]:
w, h = x
A = w * h  # area
A * L, Vmax

(9983.927658286633, 10000.0)

In [7]:
ranges = [[5., 15.], [5., 15.]]
BGA = BinaryGeneticAlgorithm(objective_fem, ranges, length=12, nPop=20, maxiter=20)
x = BGA.solve()
x = BGA.best_phenotype()
fx_fem = objective_fem(x)
x, fx_fem

(array([12.38217338,  7.77899878]), 1904.771905761849)

In [8]:
w, h = x
A = w * h  # area
A * L, Vmax

(9632.091162127794, 10000.0)

In [9]:
Wmax, fx_analytic, fx_fem

(1904.771905761849, 848.4760966743787, 1904.771905761849)

## Optimization with `scipy`

In [10]:
from scipy.optimize import minimize

x0 = [8.0, 8.0]
bnds = ((5., 15.), (5., 15.))
res = minimize(objective_analytic, x0, method='Nelder-Mead', bounds=bnds)
res.x, objective_analytic(res.x)

(array([ 5., 15.]), 1128.747795414462)

In [11]:
w, h = res.x
A = w * h  # area
A * L, Vmax

(7500.0, 10000.0)

In [12]:
x0 = [8.0, 8.0]
bnds = ((5., 15.), (5., 15.))

def objective_analytic(x):
    w, h = x
    A = w * h  # area
    Iy = w * h**3 / 12  # second moment of inertia around the y axis
    EI = E * Iy
    sol_exact = F * L**3 / (3 * EI)
    return np.abs(sol_exact * F)

cons = ({'type': 'ineq', 'fun': lambda x:  Vmax - x[0] * x[1] * L})

res = minimize(objective_analytic, x0, method='SLSQP', bounds=bnds, constraints=cons)
res.x, objective_analytic(res.x)

(array([ 6.66666667, 15.        ]), 846.5608465608369)

In [13]:
w, h = res.x
A = w * h  # area
A * L, Vmax

(10000.000000000113, 10000.0)

In [14]:
x0 = [8.0, 8.0]
bnds = ((5., 15.), (5., 15.))

def objective_fem(x):
    w, h = x
    A = w * h  # area
    Iy = w * h**3 / 12  # second moment of inertia around the y axis
    Iz = w * h**3 / 12  # second moment of inertia around the z axis
    Ix = Iy + Iz  # torsional inertia

    # model stiffness matrix
    G = E / (2 * (1 + nu))
    Hooke = np.array([
        [E*A, 0, 0, 0],
        [0, G*Ix, 0, 0],
        [0, 0, E*Iy, 0],
        [0, 0, 0, E*Iz]
    ])

    d = postproc(linsolve(build(Hooke)))
    return np.abs(global_load_vector @ d)
    
cons = ({'type': 'ineq', 'fun': lambda x:  Vmax - x[0] * x[1] * L})

res = minimize(objective_fem, x0, method='SLSQP', bounds=bnds, constraints=cons)
res.x, objective_fem(res.x)

(array([8., 8.]), 1904.771905761849)

In [15]:
w, h = res.x
A = w * h  # area
A * L, Vmax

(6400.0, 10000.0)

## Summary

In [ ]:
from sigmaepsilon.solid import BeamSection
import matplotlib.pyplot as plt
from dewloosh.mpl import triplot
mesh_params = dict(n_max=20)
section = BeamSection('CHS', d=1.0, t=0.3, n=32, mesh_params=mesh_params)
triobj = section.trimesh(T6=True).to_triobj()
fig, ax = plt.subplots(figsize=(4, 2))
triplot(triobj, fig=fig, ax=ax, lw=0.1)